In [1]:
from numpy import size
import gc
import random as random
%env XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/lib/cuda
import tensorflow as tf
from scripts import commonUtils as alex

# Allow memory growth for the GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
seed=random.randint(1,99999)
size=(100,100)
dataPath='images100'
pool=alex.getSamples(dataPath,size,batchSize=99999,seed=seed,split=0.2)
tf.keras.backend.clear_session()
poolValidation=alex.getSamples(dataPath,batchSize=9999,imgSize=size,seed=seed,training=False,split=0.2)

poolValidation.restPool()
valPool=poolValidation.getBatch(reshae=False)
batch=pool.getBatch(reshae=False)

env: XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/lib/cuda


2025-01-27 02:23:53.406631: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-27 02:23:53.432700: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-27 02:23:53.432721: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-27 02:23:53.432726: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-27 02:23:53.437385: I tensorflow/core/platform/cpu_feature_g

Found 8108 files belonging to 1 classes.
Using 6487 files for training.


2025-01-27 02:23:54.796550: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-01-27 02:23:54.815345: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-01-27 02:23:54.815487: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 8108 files belonging to 1 classes.
Using 1621 files for validation.
1621   1621
got another batch,  0  remaning  we are using  1621
6487   6487
got another batch,  0  remaning  we are using  6487


In [2]:
import numpy as np
split_arrays = np.split(batch, 3, axis=-1)
split_arraysVal=np.split(valPool, 3, axis=-1)
for i, arr in enumerate(split_arrays):
    print(f"Array {i}: shape {arr.shape}")

Array 0: shape (6487, 100, 100, 1)
Array 1: shape (6487, 100, 100, 1)
Array 2: shape (6487, 100, 100, 1)


In [3]:
class PCA(tf.keras.models.Model):
  
  def __init__(self):
    super(PCA, self).__init__()

  
    encoderModel=self.encoderCreate()
    decoderModel=self.decoderCreate()
    self.encoder = tf.keras.Model(encoderModel[0],encoderModel[1])
    self.decoder = tf.keras.Model(decoderModel[0],decoderModel[1])
  
  def encoderCreate(self):
    inputLayer=tf.keras.layers.Input(shape=(100, 100, 1))
    conv=tf.keras.layers.Conv2D(5,(3,3),activation='relu', padding='same', strides=2)(inputLayer)
    conv2=tf.keras.layers.Conv2D(2,(3,3),activation='relu', padding='same', strides=1)(conv)
    flattened = tf.keras.layers.Flatten()(conv2)
    dense=tf.keras.layers.Dense(50*50,activation="sigmoid")(flattened)
    return inputLayer,dense
 
  def decoderCreate(self):
    inputLayer=tf.keras.layers.Input(shape=(50*50))
    dense=tf.keras.layers.Dense(50*50,activation="sigmoid")(inputLayer)
    reshaped = tf.keras.layers.Reshape((50,50,1))(dense)
    upscale= tf.keras.layers.Conv2DTranspose(2, (3,3), activation='relu', padding='same', strides=2)(reshaped)
    conv1=tf.keras.layers.Conv2D(5,(3,3),activation='relu', padding='same', strides=1)(upscale)
    conv2=tf.keras.layers.Conv2D(1,(3,3),activation='sigmoid', padding='same', strides=1)(conv1)
    return  inputLayer,conv2
  
  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)

    return decoded


def ssim_loss(y_true,y_pred ):
  result=0
  for i in range(0,3):
      result+=tf.reduce_mean(tf.image.ssim(y_true[:][:][i], y_pred[:][:][i], 1.0))
  return 1.0-result/3.0

def costum_loss(y_true,y_pred ):
  loss=tf.losses.mse(y_true,y_pred)
  ssim_cal=ssim_loss(y_true,y_pred)
  return loss+ssim_cal*loss

In [4]:
primo=PCA()
#tf.keras.utils.plot_model(autoencoderRevamp.decoder, "multi_input_and_output_model.png", expand_nested=True,show_shapes=True)
opt=tf.keras.optimizers.Adam(learning_rate=0.001)
primo.compile(optimizer=opt, loss=costum_loss, metrics=['mse',ssim_loss,costum_loss])
primo.fit(split_arrays[0], split_arrays[0], # Note!
                  epochs=10,
                  shuffle=True,
                  batch_size=128,
                  validation_data=(split_arraysVal[0], split_arraysVal[0]),
               callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)])

Epoch 1/10


2025-01-27 02:24:00.451171: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2025-01-27 02:24:01.746965: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f4d0f3876a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-01-27 02:24:01.746995: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-01-27 02:24:01.750670: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-01-27 02:24:01.825205: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


51/51 [==============================] - 9s 67ms/step - loss: 0.1516 - mse: 0.0835 - ssim_loss: 0.8150 - costum_loss: 0.1516 - val_loss: 0.1460 - val_mse: 0.0808 - val_ssim_loss: 0.8081 - val_costum_loss: 0.1460
Epoch 2/10
51/51 [==============================] - 1s 24ms/step - loss: 0.1345 - mse: 0.0744 - ssim_loss: 0.8098 - costum_loss: 0.1345 - val_loss: 0.1244 - val_mse: 0.0684 - val_ssim_loss: 0.8201 - val_costum_loss: 0.1244
Epoch 3/10
51/51 [==============================] - 1s 24ms/step - loss: 0.1088 - mse: 0.0595 - ssim_loss: 0.8288 - costum_loss: 0.1088 - val_loss: 0.0932 - val_mse: 0.0508 - val_ssim_loss: 0.8361 - val_costum_loss: 0.0932
Epoch 4/10
51/51 [==============================] - 1s 25ms/step - loss: 0.0749 - mse: 0.0410 - ssim_loss: 0.8257 - costum_loss: 0.0749 - val_loss: 0.0619 - val_mse: 0.0339 - val_ssim_loss: 0.8263 - val_costum_loss: 0.0619
Epoch 5/10
51/51 [==============================] - 1s 25ms/step - loss: 0.0543 - mse: 0.0303 - ssim_loss: 0.7900 - cos

In [5]:
gc.collect()
tf.keras.backend.clear_session()

In [6]:
secondo=PCA()
opt=tf.keras.optimizers.Adam(learning_rate=0.001)
secondo.compile(optimizer=opt, loss=costum_loss, metrics=['mse',ssim_loss,costum_loss])
secondo.fit(split_arrays[1], split_arrays[1], # Note!
                  epochs=10,
                  shuffle=True,
                  batch_size=128,
                  validation_data=(split_arraysVal[1], split_arraysVal[1]),
               callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)])
gc.collect()
tf.keras.backend.clear_session()

Epoch 1/10
51/51 [==============================] - 5s 43ms/step - loss: 0.1214 - mse: 0.0669 - ssim_loss: 0.8150 - costum_loss: 0.1214 - val_loss: 0.1073 - val_mse: 0.0590 - val_ssim_loss: 0.8192 - val_costum_loss: 0.1073
Epoch 2/10
51/51 [==============================] - 1s 25ms/step - loss: 0.0960 - mse: 0.0536 - ssim_loss: 0.7921 - costum_loss: 0.0960 - val_loss: 0.0910 - val_mse: 0.0509 - val_ssim_loss: 0.7888 - val_costum_loss: 0.0910
Epoch 3/10
51/51 [==============================] - 1s 24ms/step - loss: 0.0802 - mse: 0.0450 - ssim_loss: 0.7828 - costum_loss: 0.0802 - val_loss: 0.0682 - val_mse: 0.0381 - val_ssim_loss: 0.7904 - val_costum_loss: 0.0682
Epoch 4/10
51/51 [==============================] - 1s 24ms/step - loss: 0.0607 - mse: 0.0344 - ssim_loss: 0.7639 - costum_loss: 0.0607 - val_loss: 0.0572 - val_mse: 0.0322 - val_ssim_loss: 0.7759 - val_costum_loss: 0.0572
Epoch 5/10
51/51 [==============================] - 1s 25ms/step - loss: 0.0538 - mse: 0.0305 - ssim_loss: 0

In [7]:
terzo=PCA()
opt=tf.keras.optimizers.Adam(learning_rate=0.001)
terzo.compile(optimizer=opt, loss=costum_loss, metrics=['mse',ssim_loss,costum_loss])
terzo.fit(split_arrays[2], split_arrays[2], # Note!
                  epochs=10,
                  shuffle=True,
                  batch_size=128,
                  validation_data=(split_arraysVal[2], split_arraysVal[2]),
               callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)])
gc.collect()
tf.keras.backend.clear_session()

Epoch 1/10
51/51 [==============================] - 5s 46ms/step - loss: 0.1603 - mse: 0.0847 - ssim_loss: 0.8942 - costum_loss: 0.1603 - val_loss: 0.1301 - val_mse: 0.0682 - val_ssim_loss: 0.9064 - val_costum_loss: 0.1301
Epoch 2/10
51/51 [==============================] - 1s 24ms/step - loss: 0.1264 - mse: 0.0671 - ssim_loss: 0.8829 - costum_loss: 0.1264 - val_loss: 0.1231 - val_mse: 0.0656 - val_ssim_loss: 0.8767 - val_costum_loss: 0.1231
Epoch 3/10
51/51 [==============================] - 1s 24ms/step - loss: 0.1214 - mse: 0.0651 - ssim_loss: 0.8630 - costum_loss: 0.1214 - val_loss: 0.1174 - val_mse: 0.0631 - val_ssim_loss: 0.8622 - val_costum_loss: 0.1174
Epoch 4/10
51/51 [==============================] - 1s 25ms/step - loss: 0.1116 - mse: 0.0603 - ssim_loss: 0.8494 - costum_loss: 0.1116 - val_loss: 0.1012 - val_mse: 0.0548 - val_ssim_loss: 0.8473 - val_costum_loss: 0.1012
Epoch 5/10
51/51 [==============================] - 1s 25ms/step - loss: 0.0818 - mse: 0.0452 - ssim_loss: 0

In [10]:
class MyModel(tf.keras.Model):

  def __init__(self,a,b,c):
    super(MyModel, self).__init__()
    self.a=tf.keras.models.clone_model(a)
    self.b=tf.keras.models.clone_model(b)
    self.c=tf.keras.models.clone_model(c)
    encoderModel=self.encoderCreate()
    decoderModel=self.decoderCreate()
    self.encoder = tf.keras.Model(encoderModel[0],encoderModel[1])
    self.decoder = tf.keras.Model(decoderModel[0],decoderModel[1])
  
  def encoderCreate(self):
    inputLayer=tf.keras.layers.Input(shape=(100, 100, 3))
    channel_1 = tf.keras.layers.Lambda(lambda x: x[..., 0:1], name="channel_1")(inputLayer)  # Shape: (None, 100, 100, 1)
    channel_2 = tf.keras.layers.Lambda(lambda x: x[..., 1:2], name="channel_2")(inputLayer)  # Shape: (None, 100, 100, 1)
    channel_3 = tf.keras.layers.Lambda(lambda x: x[..., 2:3], name="channel_3")(inputLayer)  # Shape: (None, 100, 100, 1)
    resA=self.a.encoder(channel_1)
    resB=self.b.encoder(channel_2)
    resC=self.b.encoder(channel_3)
    concatenated_output = tf.keras.layers.Concatenate(axis=-1, name="concatenated_output")(
        [resA, resB, resC]
    )
    return inputLayer,concatenated_output
 
  def decoderCreate(self):
    inputLayer=tf.keras.layers.Input(shape=(50*50*3))
    res=tf.keras.layers.Reshape((50*50,3))(inputLayer)
    channel_1 = tf.keras.layers.Lambda(lambda x: x[..., 0:1], name="channel_1")(res)  # Shape: (None, 100, 100, 1)
    channel_2 = tf.keras.layers.Lambda(lambda x: x[..., 1:2], name="channel_2")(res)  # Shape: (None, 100, 100, 1)
    channel_3 = tf.keras.layers.Lambda(lambda x: x[..., 2:3], name="channel_3")(res)  # Shape: (None, 100, 100, 1)
    resA=self.a.decoder(channel_1)
    resB=self.b.decoder(channel_2)
    resC=self.b.decoder(channel_3)
    concatenated_output = tf.keras.layers.Concatenate(axis=-1, name="concatenated_output")(
        [resA, resB, resC]
    )
    return  inputLayer,concatenated_output
 

  def call(self, inputs):
    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded

In [11]:
from PIL import Image
aa=primo.encoder(split_arrays[0][0:100])
bb=primo.decoder(aa)
print(bb[0])
model=MyModel(primo,secondo,terzo)
a= model.encoder(valPool[0:10])
decoder=model.decoder(a)
process_array = lambda x: (np.clip(x,0.0,1.0) * 255).astype(np.uint8)
Image.fromarray(process_array(decoder[10])).show()

tf.Tensor(
[[[0.31163484]
  [0.27632046]
  [0.37652445]
  ...
  [0.22926748]
  [0.26631057]
  [0.349089  ]]

 [[0.33756664]
  [0.28026652]
  [0.39816973]
  ...
  [0.29477805]
  [0.35898048]
  [0.3240205 ]]

 [[0.3042416 ]
  [0.27625445]
  [0.38448375]
  ...
  [0.21437399]
  [0.2660388 ]
  [0.37596902]]

 ...

 [[0.30242988]
  [0.15409774]
  [0.18486167]
  ...
  [0.13924296]
  [0.1826684 ]
  [0.3605876 ]]

 [[0.35725597]
  [0.22855563]
  [0.20902945]
  ...
  [0.11854735]
  [0.13671668]
  [0.2552229 ]]

 [[0.38214198]
  [0.2965852 ]
  [0.3024917 ]
  ...
  [0.22384237]
  [0.22052974]
  [0.35553208]]], shape=(100, 100, 1), dtype=float32)


2025-01-27 02:29:16.390581: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at strided_slice_op.cc:111 : INVALID_ARGUMENT: slice index 10 of dimension 0 out of bounds.


InvalidArgumentError: {{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:GPU:0}} slice index 10 of dimension 0 out of bounds. [Op:StridedSlice] name: strided_slice/